# Text Classification

In [121]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Binarizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')


Data collection

In [117]:
#Data collection
#extract csv file
import zipfile
with zipfile.ZipFile('archive (1).zip','r') as zip_ref:
    zip_ref.extractall()

In [118]:
#read Top_post_comments.csv file
df = pd.read_csv('Top_Posts_Comments.csv')

#read Top_posts.csv file
df_1 = pd.read_csv('Top_Posts.csv')

In [119]:
df

,post_id,comment
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,The future 🤯
2,gh1dj9,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,Ohh the nightmare of making this into a stable...
...,...,...
223169,efk5n3,LiDAR is mot powerful sensor for the auto driv...
223170,efk5n3,So it can now idenrify traffic lights? Musk pr...
223171,efk5n3,Hydranet bro!
223172,efk5n3,It even shows flashing yellow turn arrows.


In [128]:
pd.set_option('display.max_colwidth',None)
df

,post_id,comment
0,gh1dj9,"Twitter thread: [https://twitter.com/cyrildiagne/status/1259441154606669824](https://twitter.com/cyrildiagne/status/1259441154606669824)\n\nCode: [https://github.com/cyrildiagne/ar-cutpaste/tree/clipboard](https://github.com/cyrildiagne/ar-cutpaste/tree/clipboard)\n\nBackground removal is done with U^(2-Net) (Qin et Al, Pattern Recognition 2020): [https://github.com/NathanUA/U-2-Net](https://github.com/NathanUA/U-2-Net)\n\n**/!\\ EDIT:** You can now subscribe to a beta program to get early access to the app: [https://arcopypaste.app](https://arcopypaste.app) !"
1,gh1dj9,The future 🤯
2,gh1dj9,Simple yet very useful. Thank you for sharing the code.
3,gh1dj9,"Almost guaranteed, Apple will copy your idea in 3, 2, 1...."
4,gh1dj9,Ohh the nightmare of making this into a stable product... Enough to drive you mad just thinking about it
...,...,...
223169,efk5n3,LiDAR is mot powerful sensor for the auto driving cars but it cannot detect colors or signs. Another problem is current LiDARs are all based mechanical archtecture. New solid state LiDAR is necessary to used it for the mass production cars.
223170,efk5n3,So it can now idenrify traffic lights? Musk promised robotaxis by now. :)
223171,efk5n3,Hydranet bro!
223172,efk5n3,It even shows flashing yellow turn arrows.


In [5]:
df_1

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020
1,kuc6tz,[D] A Demo from 1993 of 32-year-old Yann LeCun...,MachineLearning,https://v.redd.it/25nxi9ojfha61,Discussion,5851,133,0.98,2021-01-10 10:30:36,2021
2,g7nfvb,[R] First Order Motion Model applied to animat...,MachineLearning,https://v.redd.it/rlmmjm1q5wu41,Research,4761,111,0.97,2020-04-25 04:27:23,2020
3,lui92h,[N] AI can turn old photos into moving Images ...,MachineLearning,https://v.redd.it/ikd5gjlbi8k61,News,4688,230,0.97,2021-02-28 15:12:28,2021
4,ohxnts,[D] This AI reveals how much time politicians ...,MachineLearning,https://i.redd.it/34sgziebfia71.jpg,Discussion,4568,228,0.96,2021-07-11 04:18:59,2021
...,...,...,...,...,...,...,...,...,...,...
2982,slx33m,We live in beautiful times where you can learn...,artificial,https://github.com/louisfb01/start-machine-lea...,Discussion,84,6,0.90,2022-02-06 13:50:02,2022
2983,k9otbj,Yann LeCun’s Deep Learning Course Free From NYU,artificial,https://www.i-programmer.info/news/99-professi...,News,78,1,0.97,2020-12-09 09:22:52,2020
2984,k2orib,You Can Now Learn for FREE: 9 Courses by Googl...,artificial,https://laconicml.com/free-artificial-intellig...,Self Promotion,80,2,0.95,2020-11-28 14:43:43,2020
2985,ex9w4w,"Chatbot trained on ""public domain social media...",artificial,https://ai.googleblog.com/2020/01/towards-conv...,news,80,10,0.97,2020-02-01 17:55:23,2020


In [6]:
#df_1 post_id no duplicate values (primary key)
duplicates = df_1[df_1['post_id'].duplicated()]
len(duplicates)

0

In [7]:
df.head()

,post_id,comment
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,The future 🤯
2,gh1dj9,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,Ohh the nightmare of making this into a stable...


In [8]:
df_1.head()

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020
1,kuc6tz,[D] A Demo from 1993 of 32-year-old Yann LeCun...,MachineLearning,https://v.redd.it/25nxi9ojfha61,Discussion,5851,133,0.98,2021-01-10 10:30:36,2021
2,g7nfvb,[R] First Order Motion Model applied to animat...,MachineLearning,https://v.redd.it/rlmmjm1q5wu41,Research,4761,111,0.97,2020-04-25 04:27:23,2020
3,lui92h,[N] AI can turn old photos into moving Images ...,MachineLearning,https://v.redd.it/ikd5gjlbi8k61,News,4688,230,0.97,2021-02-28 15:12:28,2021
4,ohxnts,[D] This AI reveals how much time politicians ...,MachineLearning,https://i.redd.it/34sgziebfia71.jpg,Discussion,4568,228,0.96,2021-07-11 04:18:59,2021


Data cleaning:

In [9]:
#create comment 1 coloumn with converting comment coloumn data into string
import re
con_to_str = lambda x: str(x)
df['comment1'] = df['comment'].apply(con_to_str)

In [10]:
#remove urls
url_pattern = re.compile(r'http\S+|www\S+')
def remove_urls(sentence):
    return url_pattern.sub('', sentence)
df['comment1'] = df['comment1'].apply(remove_urls)
df['comment1']

0         Twitter thread: [\n\nCode: [\n\nBackground rem...
1                                              The future 🤯
2         Simple yet very useful. Thank you for sharing ...
3         Almost guaranteed, Apple will copy your idea i...
4         Ohh the nightmare of making this into a stable...
                                ...                        
223169    LiDAR is mot powerful sensor for the auto driv...
223170    So it can now idenrify traffic lights? Musk pr...
223171                                        Hydranet bro!
223172           It even shows flashing yellow turn arrows.
223173      Ya just saw karpathy talk on hydra and pytorch.
Name: comment1, Length: 223174, dtype: object

In [11]:
#remove special charecters
df['comment1'] = df['comment1'].apply(lambda Re: Re.replace("-"," "))
df['comment1'] = df['comment1'].apply(lambda Re: re.sub(r'[^a-zA-Z\s]', '', Re))
df['comment1'] = df['comment1'].str.lower()

def clean_sentence(sentence):
    return " ".join(sentence.strip().split())
df['comment1'] = df['comment1'].apply(clean_sentence)
df['comment1']

0         twitter thread code background removal is done...
1                                                the future
2         simple yet very useful thank you for sharing t...
3            almost guaranteed apple will copy your idea in
4         ohh the nightmare of making this into a stable...
                                ...                        
223169    lidar is mot powerful sensor for the auto driv...
223170    so it can now idenrify traffic lights musk pro...
223171                                         hydranet bro
223172            it even shows flashing yellow turn arrows
223173       ya just saw karpathy talk on hydra and pytorch
Name: comment1, Length: 223174, dtype: object

In [12]:
a=0
b=df.loc[a]['comment1']
b

'twitter thread code background removal is done with u net qin et al pattern recognition edit you can now subscribe to a beta program to get early access to the app'

In [13]:
c=df.loc[a]['comment']
print(c)

Twitter thread: [https://twitter.com/cyrildiagne/status/1259441154606669824](https://twitter.com/cyrildiagne/status/1259441154606669824)

Code: [https://github.com/cyrildiagne/ar-cutpaste/tree/clipboard](https://github.com/cyrildiagne/ar-cutpaste/tree/clipboard)

Background removal is done with U^(2-Net) (Qin et Al, Pattern Recognition 2020): [https://github.com/NathanUA/U-2-Net](https://github.com/NathanUA/U-2-Net)

**/!\\ EDIT:** You can now subscribe to a beta program to get early access to the app: [https://arcopypaste.app](https://arcopypaste.app)  !


In [14]:
df

,post_id,comment,comment1
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...,twitter thread code background removal is done...
1,gh1dj9,The future 🤯,the future
2,gh1dj9,Simple yet very useful. Thank you for sharing ...,simple yet very useful thank you for sharing t...
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i...",almost guaranteed apple will copy your idea in
4,gh1dj9,Ohh the nightmare of making this into a stable...,ohh the nightmare of making this into a stable...
...,...,...,...
223169,efk5n3,LiDAR is mot powerful sensor for the auto driv...,lidar is mot powerful sensor for the auto driv...
223170,efk5n3,So it can now idenrify traffic lights? Musk pr...,so it can now idenrify traffic lights musk pro...
223171,efk5n3,Hydranet bro!,hydranet bro
223172,efk5n3,It even shows flashing yellow turn arrows.,it even shows flashing yellow turn arrows


Data transformation: 

In [15]:
#merge df and df_1
merged_df = pd.merge(df, df_1[['post_id', 'subreddit']], on='post_id', how='left')
merged_df

,post_id,comment,comment1,subreddit
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...,twitter thread code background removal is done...,MachineLearning
1,gh1dj9,The future 🤯,the future,MachineLearning
2,gh1dj9,Simple yet very useful. Thank you for sharing ...,simple yet very useful thank you for sharing t...,MachineLearning
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i...",almost guaranteed apple will copy your idea in,MachineLearning
4,gh1dj9,Ohh the nightmare of making this into a stable...,ohh the nightmare of making this into a stable...,MachineLearning
...,...,...,...,...
223169,efk5n3,LiDAR is mot powerful sensor for the auto driv...,lidar is mot powerful sensor for the auto driv...,artificial
223170,efk5n3,So it can now idenrify traffic lights? Musk pr...,so it can now idenrify traffic lights musk pro...,artificial
223171,efk5n3,Hydranet bro!,hydranet bro,artificial
223172,efk5n3,It even shows flashing yellow turn arrows.,it even shows flashing yellow turn arrows,artificial


In [16]:
merged_df['subreddit'].isnull().sum()

6

In [17]:
mode_value = merged_df['subreddit'].mode()[0]
merged_df['subreddit'] = merged_df['subreddit'].fillna(mode_value)

In [18]:
merged_df['subreddit'].isnull().sum()

0

# Navie bayes classification

Multinomial Naive Bayes

In [18]:
mer_mul=merged_df.copy()

In [19]:
#Mode imputation
mode_value = mer_mul['subreddit'].mode()[0]
mer_mul['subreddit'] = mer_mul['subreddit'].fillna(mode_value)

In [22]:
# Decide on the proportions of your data
train_size = 0.7
dev_test_size = 0.3

# Split your data into the training set and the rest of the data
train_data, remaining_data = train_test_split(mer_mul, train_size=train_size, random_state=42)

# Split the remaining data into the development set and the test set
dev_data, test_data = train_test_split(remaining_data, train_size=dev_test_size/(dev_test_size+dev_test_size),random_state=42)

# Print the sizes of each set
print(f"Training set size: {len(train_data)}")
print(f"Development set size: {len(dev_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 156221
Development set size: 33476
Test set size: 33477


In [23]:
#dev_ data for Multinomial

# split the dataset into training and testing sets
X_train=train_data['comment1']
X_test=dev_data['comment1']
y_train=train_data['subreddit']
y_test =dev_data['subreddit']

# convert the sentences in the training set into feature vectors
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)


# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7089556697335404


In [28]:
# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB(alpha=0.9)
clf.fit(X_train_vect, y_train)


# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7087764368502808


In [32]:
# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB(alpha=1.1)
clf.fit(X_train_vect, y_train)


# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7091647747640101


In [33]:
#test_data for MultinomialNB

#from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import accuracy_score

# load the dataset into a pandas dataframe

# split the dataset into training and testing sets
X_train=train_data['comment1']
X_test=test_data['comment1']
y_train=train_data['subreddit']
y_test =test_data['subreddit']

# convert the sentences in the training set into feature vectors
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)


In [36]:
# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7078889984168235


In [35]:
# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB(alpha=1.1)
clf.fit(X_train_vect, y_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7080383546912806


In [34]:
# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB(alpha=0.9)
clf.fit(X_train_vect, y_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7080980972010634


Bernoulli Naive Bayes

In [37]:
mer_bir=merged_df.copy()

In [38]:
#Mode imputation
mode_value2 = mer_bir['subreddit'].mode()[0]
mer_bir['subreddit'] = mer_bir['subreddit'].fillna(mode_value2)

In [39]:
#spliting of data into train, dev, test
# Decide on the proportions of your data
train_size = 0.7
dev_test_size = 0.3

# Split your data into the training set and the rest of the data
train_data, remaining_data = train_test_split(mer_bir, train_size=train_size, random_state=42)

# Split the remaining data into the development set and the test set
dev_data, test_data = train_test_split(remaining_data, train_size=dev_test_size/(dev_test_size+dev_test_size),random_state=42)

# Print the sizes of each set
print(f"Training set size: {len(train_data)}")
print(f"Development set size: {len(dev_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 156221
Development set size: 33476
Test set size: 33477


In [42]:
#dev Dataset for Bernoulli Naive Bayes model
X_train=train_data['comment1']
X_test=dev_data['comment1']
y_train=train_data['subreddit'].to_numpy()
y_test=dev_data['subreddit'].to_numpy()


# convert the text data into a bag-of-words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_train_bow=csr_matrix(X_train_bow)
X_test_bow = vectorizer.transform(X_test)
X_test_bow=csr_matrix(X_test_bow)



# convert the bag-of-words representation to a binary representation
binarizer = Binarizer()
X_train_bin = binarizer.fit_transform(X_train_bow)
X_test_bin = binarizer.transform(X_test_bow)

# train a Bernoulli Naive Bayes model
clf = BernoulliNB()
clf.fit(X_train_bin, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test_bin)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.6909128928187358


In [45]:
# train a Bernoulli Naive Bayes model
clf = BernoulliNB(alpha=1.5)
clf.fit(X_train_bin, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test_bin)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.7032202174692317


In [48]:
#test dataset Bernoulli Naive Bayes model
#spliting of data into train, dev, test
X_train=train_data['comment1']
X_test=test_data['comment1']
y_train=train_data['subreddit'].to_numpy()
y_test=test_data['subreddit'].to_numpy()


# convert the text data into a bag-of-words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_train_bow=csr_matrix(X_train_bow)
X_test_bow = vectorizer.transform(X_test)
X_test_bow=csr_matrix(X_test_bow)



# convert the bag-of-words representation to a binary representation
binarizer = Binarizer()
X_train_bin = binarizer.fit_transform(X_train_bow)
X_test_bin = binarizer.transform(X_test_bow)

# train a Bernoulli Naive Bayes model
clf = BernoulliNB()
clf.fit(X_train_bin, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test_bin)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)



Accuracy: 0.6917883920303491


In [56]:
# train a Bernoulli Naive Bayes model
clf = BernoulliNB(alpha=2.1)
clf.fit(X_train_bin, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test_bin)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.7038862502613735


In [49]:
# train a Bernoulli Naive Bayes model
clf = BernoulliNB(alpha=1.5)
clf.fit(X_train_bin, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test_bin)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.7052603279863787


# KNeighbors Classifier

In [28]:
mer_knn=merged_df.copy()

In [29]:
#Mode imputation
mode_value = mer_knn['subreddit'].mode()[0]
mer_knn['subreddit'] = mer_knn['subreddit'].fillna(mode_value)

In [30]:
# Decide on the proportions of your data
train_size = 0.7
dev_test_size = 0.3

# Split your data into the training set and the rest of the data
train_data, remaining_data = train_test_split(mer_knn, train_size=train_size, random_state=42)

# Split the remaining data into the development set and the test set
dev_data, test_data = train_test_split(remaining_data, train_size=dev_test_size/(dev_test_size+dev_test_size),random_state=42)

# Print the sizes of each set
print(f"Training set size: {len(train_data)}")
print(f"Development set size: {len(dev_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 156221
Development set size: 33476
Test set size: 33477


In [31]:
# split the dataset into training and testing sets
#df['column_name'] = df['column_name'].astype(float)
X_train=train_data['comment1']
X_test=dev_data['comment1']
y_train=train_data['subreddit']
y_test =dev_data['subreddit']

In [32]:
from sklearn.neighbors import KNeighborsClassifier

# Convert the text data into a matrix of token counts using CountVectorizer
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [33]:
# Train a kNN classifier on the training set
k = 5 # number of neighbors
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train_vect, y_train)

# Evaluate the accuracy of the classifier on the dev set
y_pred = clf.predict(X_test_vect)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.5230612976460748


In [34]:
# Train a kNN classifier on the training set
k = 3 # number of neighbors
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train_vect, y_train)

# Evaluate the accuracy of the classifier on the dev set
y_pred = clf.predict(X_test_vect)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.5242561835344725


In [35]:
#for test dataset
# split the dataset into training and testing sets
#df['column_name'] = df['column_name'].astype(float)
X_train=train_data['comment1']
X_test=test_data['comment1']
y_train=train_data['subreddit']
y_test =test_data['subreddit']

In [36]:
# Convert the text data into a matrix of token counts using CountVectorizer
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [37]:
# Train a kNN classifier on the training set
k = 5 # number of neighbors
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train_vect, y_train)

# Evaluate the accuracy of the classifier on the dev set
y_pred = clf.predict(X_test_vect)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.5325148609493084


In [38]:
# Train a kNN classifier on the training set
k = 3 # number of neighbors
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train_vect, y_train)

# Evaluate the accuracy of the classifier on the dev set
y_pred = clf.predict(X_test_vect)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.531051169459629


In [39]:
# Train a kNN classifier on the training set
k = 21 # number of neighbors
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train_vect, y_train)

# Evaluate the accuracy of the classifier on the dev set
y_pred = clf.predict(X_test_vect)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.5509752964722048


# Decision Tree Classifier

In [21]:
#decision tree
mer_des=merged_df.copy()
#Mode imputation
mode_value = mer_des['subreddit'].mode()[0]
mer_des['subreddit'] = mer_des['subreddit'].fillna(mode_value)

In [23]:
#split dataset into train, dev,test datasets 
# Decide on the proportions of your data
train_size = 0.7
dev_test_size = 0.3

# Split your data into the training set and the rest of the data
train_data, remaining_data = train_test_split(mer_des, train_size=train_size, random_state=42)

# Split the remaining data into the development set and the test set
dev_data, test_data = train_test_split(remaining_data, train_size=dev_test_size/(dev_test_size+dev_test_size),random_state=42)

# Print the sizes of each set
print(f"Training set size: {len(train_data)}")
print(f"Development set size: {len(dev_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 156221
Development set size: 33476
Test set size: 33477


In [24]:
#dev_ data for Decision Tree

# split the dataset into training and testing sets
X_train=train_data['comment1']
X_test=dev_data['comment1']
y_train=train_data['subreddit']
y_test =dev_data['subreddit']

# convert the sentences in the training set into feature vectors
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

# train a Naive Bayes Classifier on the feature vectors and class labels
clf = DecisionTreeClassifier()
clf.fit(X_train_vect, y_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.5870175648225594


In [26]:
#test_ data for Decision Tree

# split the dataset into training and testing sets
X_train=train_data['comment1']
X_test=test_data['comment1']
y_train=train_data['subreddit']
y_test =test_data['subreddit']

# convert the sentences in the training set into feature vectors
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

# train a Naive Bayes Classifier on the feature vectors and class labels
clf = DecisionTreeClassifier()
clf.fit(X_train_vect, y_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.5936911909669326


# RandomForestClassifier

In [19]:
mer_ram=merged_df.copy()

#Mode imputation
mode_value = mer_ram['subreddit'].mode()[0]
mer_ram['subreddit'] = mer_ram['subreddit'].fillna(mode_value)

In [20]:
# Decide on the proportions of your data
train_size = 0.7
dev_test_size = 0.3

# Split your data into the training set and the rest of the data
train_data, remaining_data = train_test_split(mer_ram, train_size=train_size, random_state=42)

# Split the remaining data into the development set and the test set
dev_data, test_data = train_test_split(remaining_data, train_size=dev_test_size/(dev_test_size+dev_test_size),random_state=42)

# Print the sizes of each set
print(f"Training set size: {len(train_data)}")
print(f"Development set size: {len(dev_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 156221
Development set size: 33476
Test set size: 33477


In [21]:
# split the dataset into training and testing sets
#df['column_name'] = df['column_name'].astype(float)
X_train=train_data['comment1']
X_test=dev_data['comment1']
y_train=train_data['subreddit']
y_test =dev_data['subreddit']

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# preprocess data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [23]:
#dev_data
# train random forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# test model accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.6823395865694826


In [24]:
# split the dataset into training and testing sets
#df['column_name'] = df['column_name'].astype(float)
X_train=train_data['comment1']
X_test=test_data['comment1']
y_train=train_data['subreddit']
y_test =test_data['subreddit']

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# preprocess data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [26]:
#test_data
# train random forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# test model accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.6848582608955402


# Using Lemmatization for multinomialNB model

In [95]:
mer_lem=merged_df.copy()

#Mode imputation
mode_value = mer_lem['subreddit'].mode()[0]
mer_lem['subreddit'] = mer_lem['subreddit'].fillna(mode_value)

In [96]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [97]:
import nltk
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [98]:
from nltk.stem import WordNetLemmatizer

# Create a WordNetLemmatizer object
lemmatizer = WordNetLemmatizer()

# Define a function to apply lemmatization on a sentence
def lemmatize_sentence(sentence):
    return ' '.join([lemmatizer.lemmatize(word) for word in sentence.split()])

# Apply lemmatization on the 'comment_stemmed' column
mer_lem['comment1'] = mer_lem['comment1'].apply(lemmatize_sentence)

In [99]:
# Decide on the proportions of your data
train_size = 0.7
dev_test_size = 0.3

# Split your data into the training set and the rest of the data
train_data, remaining_data = train_test_split(mer_lem, train_size=train_size, random_state=42)

# Split the remaining data into the development set and the test set
dev_data, test_data = train_test_split(remaining_data, train_size=dev_test_size/(dev_test_size+dev_test_size),random_state=42)

# Print the sizes of each set
print(f"Training set size: {len(train_data)}")
print(f"Development set size: {len(dev_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 156221
Development set size: 33476
Test set size: 33477


In [113]:
# split the dataset into training and testing sets
#df['column_name'] = df['column_name'].astype(float)
X_train=train_data['comment1']
X_test=dev_data['comment1']
y_train=train_data['subreddit']
y_test =dev_data['subreddit']

In [114]:
#dev data
# convert the sentences in the training set into feature vectors
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)


# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7090751583223802


In [103]:
#train data
# convert the sentences in the training set into feature vectors
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)


# make predictions on the testing set
y_pred = clf.predict(X_train_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7389083413881616


In [105]:
# split the dataset into training and testing sets
#df['column_name'] = df['column_name'].astype(float)
X_train=train_data['comment1']
X_test=test_data['comment1']
y_train=train_data['subreddit']
y_test =test_data['subreddit']

In [106]:
#test data
# convert the sentences in the training set into feature vectors
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

# convert the sentences in the testing set into feature vectors
X_test_vect = vectorizer.transform(X_test)

# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)


# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Accuracy:  0.7068136332407324


In [111]:
# train a Naive Bayes Classifier on the feature vectors and class labels
clf = MultinomialNB(alpha=1.1)
clf.fit(X_train_vect, y_train)


# make predictions on the testing set
y_pred = clf.predict(X_test_vect)

# evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.7069928607700809
